# COMP40610 Information Visualisation Assignment Data Processing

## 1. Data Selection

本次作业我选择的数据集是从CSO中的High Value Dataset 中的Labour Market Section中选取。作为即将毕业的爱尔兰国际学生，抱持着对于爱尔兰的就业市场的兴趣，想要分析近年来，在不同因素影响下爱尔兰的就业市场形势，分析爱尔兰的劳动力市场变化情况

我从[CSO High Value Datasets](https://www.cso.ie/en/statistics/highvaluedatasetshvd)中的Labour Market and Earnings 板块中下载了相关的数据集，具体原始数据包括：

 - Annual employment rate
 - Annual percentage of part-time work
 - Annual unemployment rate
 - Annual long term unemployment rate
 - Annual percentage of potential additional labour force
 - Quarterly employment rate
 - Quarterly unemployment and long-term unemployment rate

## 2. Tasks and Question Identification

根据CSO提供的数据集，我想要探讨的问题（5个）包括：
1. 对于就业率和失业率在性别问题上的差异化问题，并分析在不同的区域、不同的教育程度或年龄上，是否会有不同的差异趋势
2. 由于女性在生育或家庭中的角色问题，可能一部分女性因承担家庭责任而选择放弃全职的工作，而转而进行兼职工作。因此我们同时想要分析，在兼职率中，不同性别的差异化
3. 在爱尔兰，不同的教育程度下的教育率，兼职率，失业率，长期失业率和潜在的劳动力分别为多少？
	- 增添性别分类
4. 随着时间的变化，爱尔兰的总体就业率和失业率变化趋势,
	- 可以根据不同季度变化体现
	- 或者按照不同年份的变化体现
	- 注意：可以展现covid-19对于就业率的影响
5. 不同年龄段之内的就业模式和差率
	- 青年失业率和爱尔兰总体失业率的对比
	- 加入时间的变化

## 3. Data Cleaning

I want to anaylyse these tasks/questions so I need to conbined all five datasets together.

In [1]:
#Import the required packages
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

import os



- **Import datasets**

- Delete unneseccery column

    We inspect raw dateset and found the 'Unit' column only has one unique value. It is '%' sign to represent the unit of value. 
    
    So I decided to drop this column

In [3]:
# Define the files and their corresponding metric column names
files = {
    '../raw_datasets/ALF01_Annual Employment Rate.csv': 'Employment_Rate',
    '../raw_datasets/ALF03_Annual Unemployment Rate.csv': 'Unemployment_Rate'
}

# Common key columns for merging
key_columns = ['Year', 'Age Group', 'Sex', 'Education Attainment Level', 'NUTS 2 Region']

# Load and process the first file
first_file = True
combined_df = None

for filename, metric_name in files.items():
    # Read the CSV file
    df = pd.read_csv(filename)
    
    # Drop unnecessary columns (UNIT and Statistic Label)
    df = df.drop(['UNIT', 'Statistic Label'], axis=1)
    
    # Rename VALUE column to the metric name
    df = df.rename(columns={'VALUE': metric_name})
    
    # Convert VALUE to numeric, handling empty strings as NaN
    df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
    
    if first_file:
        # For the first file, use it as the base
        combined_df = df
        first_file = False
    else:
        # Merge subsequent files on the key columns
        combined_df = combined_df.merge(df, on=key_columns, how='outer')

print(f"Combined dataframe shape: {combined_df.shape}")
print(f"\nColumn names: {list(combined_df.columns)}")

Combined dataframe shape: (5616, 7)

Column names: ['Year', 'Age Group', 'Sex', 'Education Attainment Level', 'NUTS 2 Region', 'Employment_Rate', 'Unemployment_Rate']


- **Eliminate duplicated rows**

In [5]:
combined_df = combined_df.drop_duplicates()

- Deal with Missing values

In [6]:
print("Missing values:")
print(combined_df.isnull().sum())

Missing values:
Year                             0
Age Group                        0
Sex                              0
Education Attainment Level       0
NUTS 2 Region                    0
Employment_Rate               2964
Unemployment_Rate             4997
dtype: int64


I found the missing values for 'VALUE' column is too hgih and it would highly impact the information visualisation processing. So I decided to drop all those rows with missing value about 'VALUE'.

In [ ]:
print("Data types:")
print(combined_df.dtypes)

Data types:
Year                            int64
Age Group                      object
Sex                            object
Education Attainment Level     object
NUTS 2 Region                  object
Employment_Rate               float64
Unemployment_Rate             float64
dtype: object

Dataframe shape: (5616, 7)

Column names: ['Year', 'Age Group', 'Sex', 'Education Attainment Level', 'NUTS 2 Region', 'Employment_Rate', 'Unemployment_Rate']


In [38]:
combined_df = combined_df.dropna(subset=['VALUE'])

In [39]:
print(f"删除后的行数: {len(combined_df)}")
print(f"剩余缺失值: {combined_df['VALUE'].isnull().sum()}")

删除后的行数: 4146
剩余缺失值: 0


## 4.  Save cleaning dataset

In [41]:
combined_df.to_csv('combined_clean_data.csv', index=False)